In [1]:
from architecture import ContextMatcher
import json
import torch
import torch.nn.functional as F
import numpy as np
import nltk
data_dir = '/hdd/unsupervised-summarization/'

In [2]:
vocab = json.load(open(data_dir+'data-giga/vocab.json', 'r'))

In [3]:
matcher = ContextMatcher(vocab, data_dir+'trainedELMo', False)

In [4]:
raw = "A dozen current and former staff from different areas of the State Department and at different stages of their careers who spoke to CNN said the Ukraine controversy has exacerbated divisions between political appointees and career diplomats, many of whom say the department is being politicized in ways that undermine US ties to other countries."

In [5]:
s = [list(nltk.word_tokenize(raw))]

In [6]:
print(s)

[['A', 'dozen', 'current', 'and', 'former', 'staff', 'from', 'different', 'areas', 'of', 'the', 'State', 'Department', 'and', 'at', 'different', 'stages', 'of', 'their', 'careers', 'who', 'spoke', 'to', 'CNN', 'said', 'the', 'Ukraine', 'controversy', 'has', 'exacerbated', 'divisions', 'between', 'political', 'appointees', 'and', 'career', 'diplomats', ',', 'many', 'of', 'whom', 'say', 'the', 'department', 'is', 'being', 'politicized', 'in', 'ways', 'that', 'undermine', 'US', 'ties', 'to', 'other', 'countries', '.']]


In [7]:
# s = [['I', 'have', 'a', 'dog', '!'], ['I', 'have', 'a', 'cat', '.']]

In [8]:
vocab_inv = {a:b for b,a in vocab.items()}

In [9]:
def similarity(x, y):
    return F.cosine_similarity(x, y, -1)

In [10]:
def count_score(sourse, now, target):
    max_score = -1
    max_index = 0

    for i in range(now+1, len(sourse)):
        score = similarity(sourse[now], target)

        if max_score < 0 or score > max_score:
            max_score = score
            max_index = i
            
    return max_index, max_score

In [11]:
def summary(sourse, vocab, matcher):
    output = []
    now = -1
    while now < len(sourse)-1:

        index_list = []
        score_list = []
        for w in vocab:
            print(w, end='\r')
            output_emb = matcher.embed(torch.LongTensor([output+[w]]))[0][-1] ## could be done in batch
            index, score = count_score(sourse, now, output_emb)
            index_list.append(index)
            score_list.append(index)
        score_list = torch.FloatTensor(score_list)
        qcm = F.softmax(score_list, -1)
        k = np.argmax(score_list)

        output.append(vocab[k])
        now = index_list[k]
        print(vocab_inv[vocab[k]])
    
    output = [vocab_inv[i] for i in output]
    
    return output

In [16]:
def summary_batch(s, matcher):
    s_index = [[ vocab[w.lower()] for w in sen] for sen in s]
    s_index = torch.LongTensor(s_index)
    s_emb = matcher.embed(s_index)
    output_list = []
    for target in s_emb:
        print(target.shape)
#         vocab_list = list(range(len(vocab)))
        vocab_list = list(range(10000, 20000))
        output = summary(target, vocab_list, matcher)
        output_list.append(output)
    return output_list

In [17]:
result = summary_batch(s, matcher)

torch.Size([57, 1024])
everywhere
wrangling


In [ ]:
result